In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/housesalesprediction/kc_house_data.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(12,6))
sns.displot(data=df, x ='price' , kde=True)
# right skewed plot

In [ ]:
sns.countplot(data=df, x='bedrooms')

In [ ]:
df.corr()['price'].sort_values()[:-1]

In [ ]:
sns.scatterplot(data=df,x='sqft_living',y='price')

In [ ]:
sns.boxplot(data=df,x='bedrooms',y='price')

In [ ]:
plt.figure(figsize=(12,6))
sns.scatterplot(data=df,x='long',y='lat' , palette='plasma' , hue='price',alpha=0.2,edgecolor='black')
# price pattern doesn't look so clear because of the presence of outliers

In [ ]:
df.sort_values('price',ascending=False)

In [ ]:
len(df)*0.01

In [ ]:
non_top_1_percent = df.sort_values('price',ascending=False).iloc[216: , :]

In [ ]:
plt.figure(figsize=(12,6))
sns.scatterplot(data=non_top_1_percent,x='long',y='lat' , palette='coolwarm' , hue='price',alpha=0.2,edgecolor='black')
# now we can see the price pattern more clearly

In [ ]:
sns.boxplot(data=df,x='waterfront',y='price')

In [ ]:
df.head()

In [ ]:
df.drop('id',axis=1,inplace=True)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df['year'] = df['date'].apply(lambda x : x.year)
df['month'] = df['date'].apply(lambda x : x.month)

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=df,x='month',y='price')
# no significant difference

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=df,x='year',y='price')

In [ ]:
df.groupby('month').median()['price']

In [ ]:
df.groupby('year').median()['price']

In [ ]:
df.groupby('bedrooms').median()['price']

In [ ]:
df.drop('date',axis=1,inplace=True)

In [ ]:
df['zipcode'].nunique()

In [ ]:
df.groupby('zipcode').median()['price'].sort_values() # isn't much fruitful, will drop it

In [ ]:
df.drop('zipcode',axis=1,inplace=True)

In [ ]:
X = df.drop('price',axis=1).values
y = df['price'].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
sc = MinMaxScaler()

In [ ]:
X_train = sc.fit_transform(X_train)

In [ ]:
X_test = sc.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()

model.add(Dense(19,activation='relu')) # input layer

model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))

model.add(Dense(1)) # output layer

model.compile(optimizer='adam',loss='mse')

In [ ]:
model.fit(x=X_train,y=y_train,validation_data=(X_test,y_test),epochs=600,verbose=1)

In [ ]:
loss_df = pd.DataFrame(model.history.history)

In [ ]:
loss_df.head()

In [ ]:
loss_df.plot()

In [ ]:
pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score

In [ ]:
np.sqrt(mean_squared_error(y_test,pred))

In [ ]:
df['price'].median()

In [ ]:
mean_absolute_error(y_test,pred)

In [ ]:
explained_variance_score(y_test,pred)

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(y_test,pred)
plt.plot(y_test,y_test,'r')

In [ ]:
new_val = df.drop('price',axis=1).iloc[0].values
new_val = sc.transform(new_val.reshape(-1,19))

In [ ]:
model.predict(new_val)

In [ ]:
df['price'][0]

# pretty close

# comparing the ann model with catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
cb = CatBoostRegressor(random_state=123,iterations=10000)

In [ ]:
cb.fit(X_train,y_train,eval_set=(X_test,y_test), early_stopping_rounds=200)

In [ ]:
cb_pred = cb.predict(X_test)

In [ ]:
mean_squared_error(y_test,cb_pred)**0.5

In [ ]:
mean_absolute_error(y_test,cb_pred)

In [ ]:
explained_variance_score(y_test,cb_pred)

In [ ]:
cb.predict(new_val)

In [ ]:
df['price'][0]

# In this case, catboost outperformed our ann model